In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Load Data

In [2]:
df=pd.read_csv('rfm_data.csv')
df.head()

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location
0,8814,2023-04-11,943.31,Product C,890075,Tokyo
1,2188,2023-04-11,463.70,Product A,176819,London
2,4608,2023-04-11,80.28,Product A,340062,New York
3,2559,2023-04-11,221.29,Product A,239145,London
4,9482,2023-04-11,739.56,Product A,194545,Paris


# Understanding Data

In [3]:
# to see first five rows
df.head(5)

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location
0,8814,2023-04-11,943.31,Product C,890075,Tokyo
1,2188,2023-04-11,463.70,Product A,176819,London
2,4608,2023-04-11,80.28,Product A,340062,New York
3,2559,2023-04-11,221.29,Product A,239145,London
4,9482,2023-04-11,739.56,Product A,194545,Paris


In [4]:
# to see last five rows
df.tail(5)

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location
995,2970,2023-06-10,759.62,Product B,275284,London
996,6669,2023-06-10,941.50,Product C,987025,New York
997,8836,2023-06-10,545.36,Product C,512842,London
998,1440,2023-06-10,729.94,Product B,559753,Paris
999,4759,2023-06-10,804.28,Product D,467544,New York


In [5]:
# to see sample five rows
df.sample(5)

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location
702,5060,2023-05-23,248.74,Product A,901164,London
214,6820,2023-04-24,673.47,Product A,738720,London
239,1651,2023-04-26,923.05,Product C,449773,Tokyo
683,3879,2023-05-22,610.56,Product C,966589,London
506,8061,2023-05-09,561.42,Product D,208310,New York


In [6]:
# to see data shape
df.shape

(1000, 6)

In [7]:
# to see columns name
df.columns

Index(['CustomerID', 'PurchaseDate', 'TransactionAmount', 'ProductInformation',
       'OrderID', 'Location'],
      dtype='object')

In [8]:
# to see basic info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CustomerID          1000 non-null   int64  
 1   PurchaseDate        1000 non-null   object 
 2   TransactionAmount   1000 non-null   float64
 3   ProductInformation  1000 non-null   object 
 4   OrderID             1000 non-null   int64  
 5   Location            1000 non-null   object 
dtypes: float64(1), int64(2), object(3)
memory usage: 47.0+ KB


In [9]:
# to see basic statistics
df.describe()

,CustomerID,TransactionAmount,OrderID
count,1000.000000,1000.00000,1000.000000
mean,5554.789000,513.67781,554071.398000
std,2605.014863,286.09870,264695.448814
min,1011.000000,12.13000,100096.000000
25%,3273.000000,257.12000,313152.000000
50%,5538.000000,523.56500,564671.500000
75%,7821.750000,759.86000,783052.250000
max,9991.000000,999.44000,999695.000000


# Data Cleaning

In [10]:
# to check duplicated value
df.duplicated().sum()

0

In [11]:
# to see check null values
df.isnull().sum()

CustomerID            0
PurchaseDate          0
TransactionAmount     0
ProductInformation    0
OrderID               0
Location              0
dtype: int64

In [12]:
#date colmns\
df['PurchaseDate']=pd.to_datetime(df['PurchaseDate'])

In [18]:
from datetime import datetime

In [14]:
data=df.copy()

# Calculating RFM Values

In [20]:
df['recency'] = (datetime.now().date() - df['PurchaseDate'].dt.date).dt.days

In [23]:
frequency_df=df.groupby(['CustomerID'])['OrderID'].count().reset_index()
frequency_df.rename(columns={'OrderID':'frequency'},inplace=True)
df=df.merge(frequency_df,on='CustomerID',how='left')

In [27]:
monetary_df=df.groupby(['CustomerID'])['TransactionAmount'].sum().reset_index()
monetary_df.rename(columns={'TransactionAmount':'monetary'},inplace=True)
df=df.merge(monetary_df,on='CustomerID',how='left')

In [28]:
df.head()

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,recency,frequency,monetary
0,8814,2023-04-11,943.31,Product C,890075,Tokyo,573,1,943.31
1,2188,2023-04-11,463.70,Product A,176819,London,573,1,463.70
2,4608,2023-04-11,80.28,Product A,340062,New York,573,1,80.28
3,2559,2023-04-11,221.29,Product A,239145,London,573,1,221.29
4,9482,2023-04-11,739.56,Product A,194545,Paris,573,1,739.56


# Calculating RFM Scores

In [29]:
recency_segemnt=[5,4,3,2,1]
frequency_segment=[1,2,3,4,5]
monetary_segment=[1,2,3,4,5]

In [30]:
df['recency_score']=pd.cut(df['recency'],bins=5,labels=recency_segemnt)
df['frequency_score']=pd.cut(df['frequency'],bins=5,labels=frequency_segment)
df['monetary_score']=pd.cut(df['monetary'],bins=5,labels=monetary_segment)

In [31]:
df['recency_score']=df['recency_score'].astype(int)
df['frequency_score']=df['frequency_score'].astype(int)
df['monetary_score']=df['monetary_score'].astype(int)

In [32]:
df['rfm_score']=df['recency_score']+df['frequency_score']+df['monetary_score']

# RFM Value Segment

In [33]:
value_segment=['Low_Value','Mid_Value','High_Value']
df['value_segment']=pd.qcut(df['rfm_score'],q=3,labels=value_segment)

In [39]:
df

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,recency,frequency,monetary,recency_score,frequency_score,monetary_score,rfm_score,value_segment,Customer_Segment
0,8814,2023-04-11,943.31,Product C,890075,Tokyo,573,1,943.31,1,1,2,4,Low_Value,Potential Loyalists
1,2188,2023-04-11,463.70,Product A,176819,London,573,1,463.70,1,1,1,3,Low_Value,Potential Loyalists
2,4608,2023-04-11,80.28,Product A,340062,New York,573,1,80.28,1,1,1,3,Low_Value,Potential Loyalists
3,2559,2023-04-11,221.29,Product A,239145,London,573,1,221.29,1,1,1,3,Low_Value,Potential Loyalists
4,9482,2023-04-11,739.56,Product A,194545,Paris,573,1,739.56,1,1,2,4,Low_Value,Potential Loyalists
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2970,2023-06-10,759.62,Product B,275284,London,513,1,759.62,5,1,2,8,High_Value,Potential Loyalists
996,6669,2023-06-10,941.50,Product C,987025,New York,513,1,941.50,5,1,2,8,High_Value,Potential Loyalists
997,8836,2023-06-10,545.36,Product C,512842,London,513,1,545.36,5,1,2,8,High_Value,Potential Loyalists
998,1440,2023-06-10,729.94,Product B,559753,Paris,513,1,729.94,5,1,2,8,High_Value,Potential Loyalists


# RFM Customer Segments

In [43]:
def rfm_seg(value):
    if value >= 9:
        return "Champion"
    elif value >= 6 and value < 9:
        return 'Potential Loyalists'
    elif value >= 5 and value < 6:
        return 'At Risk Customers'
    elif value >= 3 and value < 5:
        return 'Lost'
    else:
        return 'Other'  


In [44]:
df['Customer_Segment']=df['rfm_score'].apply(rfm_seg)

In [45]:
df['Customer_Segment'].value_counts()

Potential Loyalists    503
Lost                   255
At Risk Customers      180
Champion                62
Name: Customer_Segment, dtype: int64

In [46]:
df.to_csv(r"F:/raf_analysis.csv")